In [ ]:
root_url = '/content/drive/MyDrive/DL_NLP/data'
train_path = root_url + '/train_complete_3.jsonl'
#train_path = root_url + '/extra_train.jsonl'
dev_path = root_url + '/dev_complete_3.jsonl'
test_path = root_url + '/test_complete_3.jsonl'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

     |████████████████████████████████| 312 kB 7.8 MB/s 
     |████████████████████████████████| 134 kB 45.5 MB/s 
     |████████████████████████████████| 1.1 MB 49.6 MB/s 
     |████████████████████████████████| 212 kB 64.9 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 127 kB 33.5 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |████████████████████████████████| 271 kB 61.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 6.7 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 6.5 MB 44.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset, load_metric, ClassLabel
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import transformers
import random
import pandas as pd

from pprint import pprint

import torch
import math
import time
import sys
import json
import numpy as np

In [ ]:
ending_names = ['A', 'B', 'C', 'D']
model_chkpt = "bert-base-uncased"
fake_sentence = ''
tokenizer  = AutoTokenizer.from_pretrained(model_chkpt, use_fast=True)
model = AutoModelForMultipleChoice.from_pretrained('/content/drive/MyDrive/DL_NLP/Final_project')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [ending_names.index(feature.pop(label_name)) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
def choices(example):
    for dic in example['question.choices']:
        example[dic['label']] = dic['text']
    example.pop('question.choices', None)
#    example.pop('question.stem', None)
    return example

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    pprint(df.to_html())

In [ ]:
def show_one(example):
    print(f"Context: {example['fact1']}")
    print(f"  A - {example['question.stem']} {example['A']}")
    print(f"  B - {example['question.stem']} {example['B']}")
    print(f"  C - {example['question.stem']} {example['C']}")
    print(f"  D - {example['question.stem']} {example['D']}")
    print(f"\nGround truth: option {example['label']}")    

In [ ]:
# !pip install lm_scorer

In [ ]:
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# # Load pre-trained model (weights)
# with torch.no_grad():
#         model = GPT2LMHeadModel.from_pretrained('gpt2')
#         model.eval()
# # Load pre-trained model tokenizer (vocabulary)
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
 
# def score(sentence):
#     tokenize_input = tokenizer.encode(sentence)
#     tensor_input = torch.tensor([tokenize_input])
#     loss=model(tensor_input, labels=tensor_input)[0]
#     return np.exp(loss.detach().numpy())

# import torch
# from lm_scorer.models.auto import AutoLMScorer as LMScorer
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# batch_size = 1
# scorer = LMScorer.from_pretrained("gpt2-large", device=device, batch_size=batch_size)



In [ ]:
def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["fact1"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["question.stem"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # for group in range(len(second_sentences)):
    #   max_index = 0
    #   max_loss = scorer.sentence_score(second_sentences[group][0])#score(second_sentences[group][0])
    #   print('test',group)
    #   for i in range(1,4):
    #     max_loss_curr = scorer.sentence_score(second_sentences[group][i])
    #     if max_loss_curr > max_loss:
    #       max_index = i
    #       max_loss = max_loss_curr
    #   second_sentences[group][max_index] = fake_sentence

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
facts = 1

input_files = [train_path,test_path,dev_path]
if facts == 0:
    output_files = ['train_complete_d.jsonl','test_complete_d.jsonl','dev_complete_d.jsonl']
else:
    output_files = ['train_complete_e.jsonl','test_complete_e.jsonl','dev_complete_e.jsonl']

for io in range(3):
    file_name = input_files[io]
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)       
        print(result['fact1'])
        if facts == 0:
            result['fact1'] = ''
        json_list[i] = json.dumps(result)
    file_name = output_files[io]
    fout = open(file_name,'wt')
    for i in range(len(json_list)):
        fout.write('%s\n' % json_list[i])
    fout.close()

batch_size = 16
if facts == 0:
    openbookQA = load_dataset('json', data_files={'train': 'train_complete_d.jsonl', 
                                                  'validation': 'dev_complete_d.jsonl', 
                                                  'test': 'test_complete_d.jsonl'})
else:
    openbookQA = load_dataset('json', data_files={'train': 'train_complete_e.jsonl', 
                                                  'validation': 'dev_complete_e.jsonl', 
                                                  'test': 'test_complete_e.jsonl'})
pprint(openbookQA['train'][0])

flatten = openbookQA.flatten()

updated = flatten.map(choices)
updated = updated.rename_column('answerKey', 'label')
pprint(updated['train'][0])

show_one(updated['train'][0])

examples = updated['train'][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])   

idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]    
show_one(updated['train'][idx])

encoded_datasets = updated.map(preprocess_function, batched=True)

model_name = model_chkpt.split("/")[-1]
args = TrainingArguments(f"{model_name}-finetuned-swag",
                          evaluation_strategy = "epoch",
                          learning_rate=5e-5,
                          per_device_train_batch_size=batch_size,
                          num_train_epochs=3,
                          weight_decay=0.01)

accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]
show_one(updated["train"][8])

trainer = Trainer(model,
                  args,
                  train_dataset=encoded_datasets["train"],
                  eval_dataset=encoded_datasets["validation"],
                  tokenizer=tokenizer,
                  data_collator=DataCollatorForMultipleChoice(tokenizer),
                  compute_metrics=compute_metrics)

trainer.train()
print('\n\n\n\n')
print('test set:')
print('\n\n\n\n')
final_eval = trainer.evaluate(eval_dataset=encoded_datasets['test'])
print(final_eval)

Streaming output truncated to the last 5000 lines.
a thermometer is used to measure temperature
a compass is used to navigate seas
An example of conservation is avoiding waste
if a habitat is removed then that habitat is destroyed
moving changes position
contact between rocks over long periods of time causes rocks to smooth
refracting sunlight causes light to split into different colors
precipitation is when water falls from the sky
a chloroplast contains chlorophyll
the Earth rotates on its axis on its axis
an animal requires nutrients for survival
eagles eat rabbits
cool temperatures cause animals to shiver
planting trees has a positive impact on an ecosystem
decomposition increases the amount of nutrients in the soil
reproduction is when an organism passes genetic information from itself to its offspring
as the amount of food an animal eats increases , the weight of that animal will increase
matter is made of molecules
earthquakes cause rock layers to fold on top of each other
plant

Using custom data configuration default-859c7ee2d9bf633b


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-859c7ee2d9bf633b/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


...

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A. If question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4957
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 930


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.171142,0.504000
2,1.067100,1.201819,0.534000
3,1.067100,1.475699,0.530000


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A. If question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-swag/checkpoint-500
Configuration saved in bert-base-uncased-finetuned-swag/checkpoint-500/config.json
Model weights saved in bert-base-uncased-finetuned-swag/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-swag/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-swag/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForMulti






test set:







{'eval_loss': 1.5072331428527832, 'eval_accuracy': 0.5180000066757202, 'eval_runtime': 11.4195, 'eval_samples_per_second': 43.785, 'eval_steps_per_second': 5.517, 'epoch': 3.0}


In [ ]:
facts = 0

input_files = [train_path,test_path,dev_path]
if facts == 0:
    output_files = ['train_complete_d.jsonl','test_complete_d.jsonl','dev_complete_d.jsonl']
else:
    output_files = ['train_complete_e.jsonl','test_complete_e.jsonl','dev_complete_e.jsonl']

for io in range(3):
    file_name = input_files[io]
    with open(file_name) as json_file:
        json_list = list(json_file)
    for i in range(len(json_list)):
        json_str = json_list[i]
        result = json.loads(json_str)       
        print(result['fact1'])
        if facts == 0:
            result['fact1'] = ''
        json_list[i] = json.dumps(result)
    file_name = output_files[io]
    fout = open(file_name,'wt')
    for i in range(len(json_list)):
        fout.write('%s\n' % json_list[i])
    fout.close()

batch_size = 16
if facts == 0:
    openbookQA = load_dataset('json', data_files={'train': 'train_complete_d.jsonl', 
                                                  'validation': 'dev_complete_d.jsonl', 
                                                  'test': 'test_complete_d.jsonl'})
else:
    openbookQA = load_dataset('json', data_files={'train': 'train_complete_e.jsonl', 
                                                  'validation': 'dev_complete_e.jsonl', 
                                                  'test': 'test_complete_e.jsonl'})
pprint(openbookQA['train'][0])

flatten = openbookQA.flatten()

updated = flatten.map(choices)
updated = updated.rename_column('answerKey', 'label')
pprint(updated['train'][0])

show_one(updated['train'][0])

examples = updated['train'][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])   

idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]    
show_one(updated['train'][idx])

encoded_datasets = updated.map(preprocess_function, batched=True)

model_name = model_chkpt.split("/")[-1]
args = TrainingArguments(f"{model_name}-finetuned-swag",
                          evaluation_strategy = "epoch",
                          learning_rate=5e-5,
                          per_device_train_batch_size=batch_size,
                          num_train_epochs=3,
                          weight_decay=0.01)

accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]
show_one(updated["train"][8])

trainer = Trainer(model,
                  args,
                  train_dataset=encoded_datasets["train"],
                  eval_dataset=encoded_datasets["validation"],
                  tokenizer=tokenizer,
                  data_collator=DataCollatorForMultipleChoice(tokenizer),
                  compute_metrics=compute_metrics)

trainer.train()
print('\n\n\n\n')
print('test set:')
print('\n\n\n\n')
final_eval = trainer.evaluate(eval_dataset=encoded_datasets['test'])
print(final_eval)

Streaming output truncated to the last 5000 lines.
a thermometer is used to measure temperature
a compass is used to navigate seas
An example of conservation is avoiding waste
if a habitat is removed then that habitat is destroyed
moving changes position
contact between rocks over long periods of time causes rocks to smooth
refracting sunlight causes light to split into different colors
precipitation is when water falls from the sky
a chloroplast contains chlorophyll
the Earth rotates on its axis on its axis
an animal requires nutrients for survival
eagles eat rabbits
cool temperatures cause animals to shiver
planting trees has a positive impact on an ecosystem
decomposition increases the amount of nutrients in the soil
reproduction is when an organism passes genetic information from itself to its offspring
as the amount of food an animal eats increases , the weight of that animal will increase
matter is made of molecules
earthquakes cause rock layers to fold on top of each other
plant

Using custom data configuration default-ceab9989c12f0f43


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ceab9989c12f0f43/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


...

Context: 
  A - As a car approaches you in the night 
  B - As a car approaches you in the night the headlights recede into the dark
  C - As a car approaches you in the night the headlights remain at a constant
  D - As a car approaches you in the night the headlights turn off

Ground truth: option A


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.429642,0.432000
2,0.643400,1.982435,0.422000
3,0.643400,2.821396,0.422000


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A. If question.stem, id, clarity, D, C, B, turkIdAnonymized, fact1, humanScore, A are not expected by `BertForMultipleChoice.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to bert-base-uncased-finetuned-swag/checkpoint-500
Configuration saved in bert-base-uncased-finetuned-swag/checkpoint-500/config.json
Model weights saved in bert-base-uncased-finetuned-swag/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-swag/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-swag/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForMulti






test set:







{'eval_loss': 2.8835136890411377, 'eval_accuracy': 0.45399999618530273, 'eval_runtime': 8.6432, 'eval_samples_per_second': 57.849, 'eval_steps_per_second': 7.289, 'epoch': 3.0}


In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5
creating metadata file for /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/775efbdc2152093295bc5824dee96da82a5f3c1f218dfface1b8cef3094bdf8f.c719a806caef7d36ec0185f14b3b5fa727d919f924abe35622b4b7147bfbb8c7.h5
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.

In [ ]:
!cp /content/test_complete_e.jsonl /content/drive/MyDrive/DL_NLP/data/